In [28]:
import numpy as np
import pandas as pd
import math

In [2]:
x=np.array([['sunny', 85.0, 85.0, 'false', 'no'], ['sunny', 80.0, 90.0, 'true', 'no'],
        ['overcast', 83.0, 86.0, 'false', 'yes'], ['rainy', 70.0, 96.0, 'false', 'yes'],
        ['rainy', 68.0, 80.0, 'false', 'yes'], ['rainy', 65.0, 70.0, 'true', 'no'],
        ['overcast', 64.0, 65.0, 'true', 'yes'], ['sunny', 72.0, 95.0, 'false', 'no'],
        ['sunny', 69.0, 70.0, 'false', 'yes'], ['rainy', 75.0, 80.0, 'false', 'yes'],
        ['sunny', 75.0, 70.0, 'true', 'yes'], ['overcast', 81.0, 75.0, 'false', 'yes'],
        ['overcast', 72.0, 90.0, 'true', 'yes'], ['rainy', 71.0, 91.0, 'true', 'no']])

In [3]:
x

array([['sunny', '85.0', '85.0', 'false', 'no'],
       ['sunny', '80.0', '90.0', 'true', 'no'],
       ['overcast', '83.0', '86.0', 'false', 'yes'],
       ['rainy', '70.0', '96.0', 'false', 'yes'],
       ['rainy', '68.0', '80.0', 'false', 'yes'],
       ['rainy', '65.0', '70.0', 'true', 'no'],
       ['overcast', '64.0', '65.0', 'true', 'yes'],
       ['sunny', '72.0', '95.0', 'false', 'no'],
       ['sunny', '69.0', '70.0', 'false', 'yes'],
       ['rainy', '75.0', '80.0', 'false', 'yes'],
       ['sunny', '75.0', '70.0', 'true', 'yes'],
       ['overcast', '81.0', '75.0', 'false', 'yes'],
       ['overcast', '72.0', '90.0', 'true', 'yes'],
       ['rainy', '71.0', '91.0', 'true', 'no']], dtype='<U8')

In [5]:
df = pd.DataFrame(x, columns = ['outlook','temperature','humidity', 'windy', 'play'])

In [6]:
df

,outlook,temperature,humidity,windy,play
0,sunny,85.0,85.0,false,no
1,sunny,80.0,90.0,true,no
2,overcast,83.0,86.0,false,yes
3,rainy,70.0,96.0,false,yes
4,rainy,68.0,80.0,false,yes
5,rainy,65.0,70.0,true,no
6,overcast,64.0,65.0,true,yes
7,sunny,72.0,95.0,false,no
8,sunny,69.0,70.0,false,yes
9,rainy,75.0,80.0,false,yes


In [294]:
class Node:
    def __init__(self, info, data, attribute_name, attribute_value):
        self.list=[] #Child
        self.info = info #Info
        self.predict = None #Predict
        self.data = data #Current data set
        self.attribute_name = attribute_name
        self.attribute_value = attribute_value
        self.smaller = None
        self.greater = None
        self.parent = None
    
    def search(self, value):
        for i in self.list:
            if i.data == value:
                return i
        return None
    
    def find_parent(self):
        return self.parent
    
    def print_node(self):
        print('Split On:', self.attribute_name, ';Parent Attribute:', self.parent.attribute_name, ';Parent Attribute Value:', self.parent.attribute_value)
        print('Split With Attribute Value:', self.attribute_value, 'Information Gain:', self.info)
        print(self.data)
        print("-------------------------------------------")
    
    def print_root(self):
        print('This is ROOT with information:', self.info)
        print("-------------------------------------------")
        
    def print_winner(self):
        print('Prediction:', self.predict)
        self.print_node()
        print("*******************************************")
        print("*******************************************")

In [266]:
class Tree:
    def __init__(self, node):
        self.root=node
        
    def createNode(self, info, data, attribute_name, attribute_value):
        return Node(info, data, attribute_name, attribute_value)

    def insert(self, node, info, data, attribute_name, attribute_value):
        if node is None:
            return self.createNode(info, data, None, None)
        else:
            x = Node(info, data, attribute_name, attribute_value)
            node.list.append(x)
            x.parent = node
            node = x
        return node

    def print_list(self,root):
        print(root.data)
        if root.list != []:
            if len(root.list)>1:
                print('start branch')
            for i in root.list:
                self.print_list(i)
        else:
            print('end branch')
    
    def print_while_remove(self,root):
        x= root
        while x.list != []:
            if (x.list[0].list == []):
                x.list[0].count = x.list[0].count-1
                print(x.list[0].data)
                if (x.list[0].count == 0):
                    self.remove(x,0)
                print("End of branch")
                x=root
            else:
                x.list[0].count = x.list[0].count-1
                print(x.list[0].data)
                y = x.list[0]
                if (x.list[0].count == 0):
                    self.remove(x,0)
                x=y
                    
            
    
    def remove(self,node,index):
        node.list.pop(0)
    
    def find_leaf(self, name):
        return self.my_dict.get(name)

In [267]:
def get_attribute_discrete_value_table(table, attribute, attribute_value):
    temp = table.loc[table[attribute]==attribute_value]
    temp = temp.drop([attribute], axis=1)
    return temp

In [268]:
def get_attribute_continuous_value_table(table, attribute, attribute_value, side):
    if side == 'left':
        temp = table.loc[table[attribute].astype(float)<attribute_value]
    else:
        temp = table.loc[table[attribute].astype(float)>attribute_value]
    return temp

In [269]:
def calculate_info(table, label):
    sum = 0
    for i in table[label].value_counts():
        freq = i/len(table.index)
        sum = sum + freq * math.log2(freq)
    return -sum

In [270]:
def calculate_expected_info(table, attribute, label):
    sum = 0
    split = None
    left_split = None
    right_split = None
    if table[attribute].nunique()<=3: #discrete
        for i in table[attribute].unique():
            temp = get_attribute_discrete_value_table(table,attribute,i)
            x = calculate_info(temp, label)
            freq = (table[attribute] == i).sum() / len(table.index)
            sum = sum+freq*x
    else: #continuous
        sum = math.inf
        for index, elem in enumerate(np.sort(table[attribute].unique().astype(np.float))):
            if index+1 < len(np.sort(table[attribute].unique().astype(np.float))):
                i = elem
                j = np.sort(table[attribute].unique().astype(np.float))[index+1]
                test = (i+j)/2
                left = get_attribute_continuous_value_table(table, attribute, test, 'left')
                right = get_attribute_continuous_value_table(table, attribute, test, 'right')
                temp = len(left.index)/len(table.index)*calculate_info(left, label) + len(right.index)/len(table.index)*calculate_info(right, label)
                if temp < sum:
                    sum = temp
                    split = test
                    left_split = left
                    right_split = right
    return [sum,split,left_split,right_split]

In [271]:
def calculate_gain(table,attribute,label):
    return calculate_info(table,label)-calculate_expected_info(table,attribute,label)[0]

In [272]:
def choose_attribute(table, attributes, label):
    m = 0
    next_to_split = None
    for i in attributes:
        if calculate_gain(table,i,label)>m:
            m = calculate_gain(table,i,label)
            next_to_split = i
    return next_to_split

In [295]:
def distribute(tree, node, table, label):
    if node == tree.root:
        node.print_root()
    if table[label].nunique()==1:
        node.predict=table[label].unique()[0]
        node.print_winner()
        return
    else:
        attribute = choose_attribute(table, list(table.columns)[:-1], label)
        if table[attribute].nunique()<=3 and table[attribute].nunique()>1:
            for i in table[attribute].unique():
                t3 = get_attribute_discrete_value_table(table, attribute, i)
                node3 = tree.insert(node, calculate_gain(table,attribute,label), t3, attribute, i)
                node3.print_node()
                distribute(tree, node3, get_attribute_discrete_value_table(table, attribute, i), label)
        elif table[attribute].nunique()> 3:
            t1 = calculate_expected_info(table, attribute, label)[2]
            node1 = tree.insert(node, calculate_gain(table,attribute,label), t1, attribute , calculate_expected_info(table, attribute, label)[1])
            node1.smaller = 1
            node1.print_node()
            distribute(tree,node1,calculate_expected_info(table, attribute, label)[2], label)
            t2 = calculate_expected_info(table, attribute, label)[3]
            node2 = tree.insert(node, calculate_gain(table,attribute,label), t2,attribute ,calculate_expected_info(table, attribute, label)[1])
            node2.greater = 1
            node2.print_node()
            distribute(tree,node2,calculate_expected_info(table, attribute, label)[3], label)
        else:
            return
                

### Build Information Gain Tree

In [296]:
tree = Tree(Node(calculate_info(df, 'play'), df, None, 'ROOT'))
distribute(tree,tree.root,df,'play')

This is ROOT with information: 0.9402859586706311
-------------------------------------------
Split On: outlook ;Parent Attribute: None ;Parent Attribute Value: ROOT
Split With Attribute Value: sunny Information Gain: 0.24674981977443933
   temperature humidity  windy play
0         85.0     85.0  false   no
1         80.0     90.0   true   no
7         72.0     95.0  false   no
8         69.0     70.0  false  yes
10        75.0     70.0   true  yes
-------------------------------------------
Split On: humidity ;Parent Attribute: outlook ;Parent Attribute Value: sunny
Split With Attribute Value: 77.5 Information Gain: 0.9709505944546686
   temperature humidity  windy play
8         69.0     70.0  false  yes
10        75.0     70.0   true  yes
-------------------------------------------
Prediction: yes
Split On: humidity ;Parent Attribute: outlook ;Parent Attribute Value: sunny
Split With Attribute Value: 77.5 Information Gain: 0.9709505944546686
   temperature humidity  windy play
8   

#### Predict Via Information Gain Model

In [333]:
def prediction(node, dict):
    if node.predict is not None:
        print(node.predict)
        return
    else:
        for i in dict:
            for j in node.list:
                if j.smaller is None and j.greater is None:
                    if i == j.attribute_name and dict.get(i) == j.attribute_value:
                        node = j
                        return prediction(node, dict)
                else:
                    if i==j.attribute_name:
                        if dict.get(i)>j.attribute_value and j.greater == 1:
                            node = j
                            prediction(node,dict)
                        elif dict.get(i)<j.attribute_value and j.smaller == 1:
                            node = j
                            return prediction(node,dict)
                    

### Result From Information Gain Model

In [335]:
dict={'outlook':'overcast','temperature':60,'humidity':62,'windy':'false'}
prediction(tree.root,dict)

yes


In [389]:
def calculate_gain_ratio(table, attribute, label):
    sum1 = 0
    sum2 = 0
    gain_ratio = 0
    split = None
    left_split = None
    right_split = None
    if table[attribute].nunique()<=3: #discrete
        for i in table[attribute].unique():
            temp = get_attribute_discrete_value_table(table,attribute,i)
            x = calculate_info(temp, label)
            freq = (table[attribute] == i).sum() / len(table.index)
            sum1 = sum1+freq*x
            freq = (table[attribute] == i).sum() / len(table.index)
            sum2 = sum2+freq*math.log2(freq)
        print(sum1)
        print(-sum2)
        gain_ratio = (calculate_info(table,label)-sum1)/(-sum2)
    else: #continuous
        for index, elem in enumerate(np.sort(table[attribute].unique().astype(np.float))):
            if index+1 < len(np.sort(table[attribute].unique().astype(np.float))):
                i = elem
                j = np.sort(table[attribute].unique().astype(np.float))[index+1]
                test = (i+j)/2
                left = get_attribute_continuous_value_table(table, attribute, test, 'left')
                right = get_attribute_continuous_value_table(table, attribute, test, 'right')
                temp1 = len(left.index)/len(table.index)*calculate_info(left, label) + len(right.index)/len(table.index)*calculate_info(right, label)
                temp2 = len(left.index)/len(table.index)*math.log2(len(left.index)/len(table.index)) + len(right.index)/len(table.index)*math.log2(len(right.index)/len(table.index))
                temp3 = (calculate_info(table,label)-temp1)/(-temp2)
                if gain_ratio < temp3:
                    gain_ratio = temp3
                    split = test
                    left_split = left
                    right_split = right
    return [gain_ratio,split,left_split,right_split]

In [390]:
def choose_attribute(table, attributes, label):
    m = 0
    next_to_split = None
    for i in attributes:
        if calculate_gain_ratio(table,i,label)[0]>m:
            m = calculate_gain_ratio(table,i,label)[0]
            next_to_split = i
    return next_to_split

In [391]:
def distribute2(tree, node, table, label):
    if node == tree.root:
        node.print_root()
    if table[label].nunique()==1:
        node.predict=table[label].unique()[0]
        node.print_winner()
        return
    else:
        attribute = choose_attribute(table, list(table.columns)[:-1], label)
        if table[attribute].nunique()<=3 and table[attribute].nunique()>1:
            for i in table[attribute].unique():
                t3 = get_attribute_discrete_value_table(table, attribute, i)
                node3 = tree.insert(node, calculate_gain_ratio(table,attribute,label)[0], t3, attribute, i)
                node3.print_node()
                distribute(tree, node3, get_attribute_discrete_value_table(table, attribute, i), label)
        elif table[attribute].nunique()> 3:
            t1 = calculate_gain_ratio(table, attribute, label)[2]
            node1 = tree.insert(node, calculate_gain_ratio(table,attribute,label)[0], t1, attribute , calculate_gain_ratio(table, attribute, label)[1])
            node1.smaller = 1
            node1.print_node()
            distribute(tree,node1,calculate_gain_ratio(table, attribute, label)[2], label)
            t2 = calculate_gain_ratio(table, attribute, label)[3]
            node2 = tree.insert(node, calculate_gain_ratio(table,attribute,label)[0], t2,attribute ,calculate_gain_ratio(table, attribute, label)[1])
            node2.greater = 1
            node2.print_node()
            distribute(tree,node2,calculate_gain_ratio(table, attribute, label)[3], label)
        else:
            return
                

### Build Gain Ratio Tree

In [392]:
tree = Tree(Node(calculate_info(df, 'play'), df, None, 'ROOT'))
distribute2(tree,tree.root,df,'play')

This is ROOT with information: 0.9402859586706311
-------------------------------------------
0.6935361388961918
1.5774062828523454
0.6935361388961918
1.5774062828523454
0.8921589282623617
0.9852281360342516
Split On: temperature ;Parent Attribute: None ;Parent Attribute Value: ROOT
Split With Attribute Value: 84.0 Information Gain: 0.30547141518417825
     outlook temperature humidity  windy play
1      sunny        80.0     90.0   true   no
2   overcast        83.0     86.0  false  yes
3      rainy        70.0     96.0  false  yes
4      rainy        68.0     80.0  false  yes
5      rainy        65.0     70.0   true   no
6   overcast        64.0     65.0   true  yes
7      sunny        72.0     95.0  false   no
8      sunny        69.0     70.0  false  yes
9      rainy        75.0     80.0  false  yes
10     sunny        75.0     70.0   true  yes
11  overcast        81.0     75.0  false  yes
12  overcast        72.0     90.0   true  yes
13     rainy        71.0     91.0   true   no
-

<ipython-input-389-911a8e961b90>:18: RuntimeWarning: invalid value encountered in double_scalars
  gain_ratio = (calculate_info(table,label)-sum1)/(-sum2)



-------------------------------------------
Prediction: yes
Split On: temperature ;Parent Attribute: humidity ;Parent Attribute Value: 88.0
Split With Attribute Value: 66.5 Information Gain: 0.2935644431995964
     outlook temperature humidity  windy play
2   overcast        83.0     86.0  false  yes
4      rainy        68.0     80.0  false  yes
8      sunny        69.0     70.0  false  yes
9      rainy        75.0     80.0  false  yes
10     sunny        75.0     70.0   true  yes
11  overcast        81.0     75.0  false  yes
-------------------------------------------
*******************************************
*******************************************
Split On: humidity ;Parent Attribute: temperature ;Parent Attribute Value: 84.0
Split With Attribute Value: 88.0 Information Gain: 0.18254790807563637
     outlook temperature humidity  windy play
1      sunny        80.0     90.0   true   no
3      rainy        70.0     96.0  false  yes
7      sunny        72.0     95.0  false   no


### Predict Via Gain Ratio Model

In [395]:
dict={'outlook':'overcast','temperature':60,'humidity':62,'windy':'false'}
prediction(tree.root,dict)

yes
